# Evaluacion del Modelo

In [ ]:
import pandas as pd
import numpy as np
import joblib
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Cargar el Modelo Entrenado
model = load_model('models/cltv_model_v1.keras')

# 2. Cargar Datos Matemáticos (Test Set)
X_test_scaled = joblib.load('data/processed/X_test_scaled.pkl')
y_test = joblib.load('data/processed/y_test.pkl')

# 3. Cargar Datos de Contexto (IDs y DataFrame original)
df_model_rolling = pd.read_parquet('data/processed/df_model_rolling.parquet')
test_ids = joblib.load('data/processed/test_ids.pkl')

print("Todo cargado. Estamos listos para auditar al modelo.")

In [ ]:
# 1. Generar Predicciones (Escala Logarítmica)
y_pred_log = model.predict(X_test_scaled)

# 2. Revertir la Transformación Logarítmica (Volver a Dinero Real)
# flatten() es para convertir la matriz [[val], [val]] a un array simple [val, val]
y_pred_real = np.expm1(y_pred_log.flatten())
y_test_real = np.expm1(y_test)

# Verificación rápida
print(f"Ejemplo Real (Test): ${y_test_real[0]:.2f}")
print(f"Ejemplo Predicho:    ${y_pred_real[0]:.2f}")

In [ ]:
# Calcular métricas en la escala real
mae = mean_absolute_error(y_test_real, y_pred_real)
rmse = np.sqrt(mean_squared_error(y_test_real, y_pred_real))
r2 = r2_score(y_test_real, y_pred_real)

print(f"\nResultados Finales del Modelo:")
print(f"------------------------------")
print(f"R2 Score (Precisión Global): {r2:.4f} (Ideal > 0.6)")
print(f"MAE (Error Promedio):       ${mae:.2f}")
print(f"RMSE (Error Cuadrático):    ${rmse:.2f}")

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test_real, y=y_pred_real, alpha=0.5)

# Línea de perfección (Diagonal)
max_val = max(y_test_real.max(), y_pred_real.max())
plt.plot([0, max_val], [0, max_val], '--r', label='Predicción Perfecta')

plt.title('CLTV Real vs. Predicho (Set de Prueba)')
plt.xlabel('Valor Real (Dinero)')
plt.ylabel('Valor Predicho por IA (Dinero)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 1. Recuperar el DataFrame de los clientes de Test
# Usamos los test_ids que guardamos para filtrar el DF original
df_test_results = df_model_rolling[df_model_rolling['customer_id'].isin(test_ids)].copy()

# Nota: df_test_results puede tener más filas que y_pred_real si no filtramos exactamente igual que en el split.
# Para evitar errores de índice, lo más seguro es reconstruir un DF simple con las predicciones:

results_df = pd.DataFrame({
    'Real_CLTV': y_test_real,
    'Predicted_CLTV': y_pred_real,
    'Error_Absoluto': np.abs(y_test_real - y_pred_real)
})

# Añadimos una columna de "Segmento" basada en la predicción
results_df['Predicted_Segment'] = pd.qcut(results_df['Predicted_CLTV'], q=3, labels=['Bajo Valor', 'Medio Valor', 'Alto Valor'])

print("Top 5 Clientes con Mayor Valor Predicho:")
print(results_df.sort_values('Predicted_CLTV', ascending=False).head())